In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
data = pd.read_csv('first_df_with_RM.csv', parse_dates=["time"])

In [3]:
data

,time,ticker,open,high,low,close,volume,time.1,rv,gamma,RM
0,2019-09-30 07:00:00,ATVI,52.117346,52.117346,51.988808,51.988808,698.0,2019-09-30 07:00:00,NaN,NaN,0.000000
1,2019-09-30 07:30:00,ATVI,52.058021,52.058021,52.058021,52.058021,199.0,2019-09-30 07:30:00,0.004790,NaN,0.000000
2,2019-09-30 08:00:00,ATVI,52.058021,52.058021,52.058021,52.058021,199.0,2019-09-30 08:00:00,0.000000,0.000000,NaN
3,2019-09-30 08:30:00,ATVI,51.899820,51.899820,51.899820,51.899820,132.0,2019-09-30 08:30:00,0.025028,-0.000000,0.000000
4,2019-09-30 09:00:00,ATVI,52.107360,52.107360,52.107360,52.107360,150.0,2019-09-30 09:00:00,0.043073,-0.032833,-0.032833
...,...,...,...,...,...,...,...,...,...,...,...
20122,2021-09-17 16:00:00,ATVI,79.355000,79.830000,79.295000,79.560000,1853014.0,2021-09-17 16:00:00,0.040000,0.062000,0.062000
20123,2021-09-17 16:30:00,ATVI,79.560000,79.560000,79.560000,79.560000,692564.0,2021-09-17 16:30:00,0.000000,0.000000,0.000000
20124,2021-09-17 17:00:00,ATVI,79.000000,79.020000,79.000000,79.010000,1236.0,2021-09-17 17:00:00,0.302500,-0.000000,0.000000
20125,2021-09-17 17:30:00,ATVI,79.200000,79.200000,79.200000,79.200000,201.0,2021-09-17 17:30:00,0.036100,-0.104500,-0.104500


In [4]:
from scipy.optimize import minimize

In [24]:
def log_Q2(alphaR, betaR, rv, RM):
    T = len(rv)
    mu_t = T**0.5 * np.sum/(RM[:int(T**0.5)])
    ll = 0
    
    mu = rv.mean()
    muR = RM.mean()
    k = mu / muR
    for i in range(1, T):
        mu_t = muR*(1 - alphaR - betaR) + alphaR*RM[i] + betaR*mu_t
        ll += 0.5*(np.log(mu_t) + RM[i]/mu_t)
        
    return ll

In [18]:
def log_Q1(alpha, beta, rv, RM):
    T = len(rv)
    h = T**0.5 * np.sum(rv[:int(T**0.5)])
    ll = 0
    
    mu = rv.mean()
    muR = RM.mean()
    k = mu / muR
    for i in range(1, T):
        h = mu*(1 - alpha*k - beta) + alpha*RM[i-1] + beta*h
        ll += 0.5*(np.log(h) + rv[i]/h)
    
    print(alpha, beta)
    return ll

In [19]:
df = data[data['time'].dt.date == datetime.datetime(2019,9, 30).date()]
rv = df['rv'].fillna(0)
RM = df['RM'].fillna(0)

In [20]:
fun = lambda params: log_Q1(params[0], params[1], rv, RM)
bnds = ((0, None), (0, 1))
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] + x[1] - 1})
res = minimize(fun, (0.3, 0.5), method='SLSQP', bounds=bnds, tol=1e-6, constraints=cons)

0.3 0.5
0.3000000149011612 0.5
0.3 0.5000000149011612
4.273192910630996e-12 1.0
0.23328503725813704 0.6111916045713555
0.23328505215929823 0.6111916045713555
0.23328503725813704 0.6111916194725167
0.0 1.0
0.16204555422074207 0.7299240762998634
0.16204556912190327 0.7299240762998634
0.16204555422074207 0.7299240912010246
3.041705776141157e-12 0.9999999999970723
0.11440260183015277 0.8093289969511519
0.11440261673131397 0.8093289969511519
0.11440260183015277 0.8093290118523131
0.13346205847119977 0.8665379415266834
0.13346207337236096 0.8665379415266834
0.13346205847119977 0.8665379564278446
0.1737077298522413 0.8262922697248717
0.1737077447534025 0.8262922697248717
0.1737077298522413 0.8262922846260329
0.25894105949195456 0.7410589405190704
0.25894107439311576 0.7410589405190704
0.25894105949195456 0.7410589554202316
0.3439795940252848 0.6560204059594821
0.34397960892644597 0.6560204059594821
0.3439795940252848 0.6560204208606433
0.46571514035888784 0.5342848596473362
0.4657151552600490

In [21]:
res

     fun: -23.977179138560135
     jac: array([7.2856813 , 7.88429642])
 message: 'Optimization terminated successfully'
    nfev: 39
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([1.00000000e+00, 9.97969971e-11])

In [23]:
fun(res.x)

0.9999999998965988 9.979699705509049e-11


-23.977179138560135